You can download the data at https://cloud.zerynth.com/wks-6sdnjgqfpv98/exports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
INPUT_FILENAME = 'example.csv'

df = pd.read_csv(INPUT_FILENAME)

In [4]:
df.head()

,timestamp_device,timestamp_in,workspace_id,fleet_id,device_id,tag,n_devices,scans,sum_rssi,ts
0,2022-06-10 18:03:50+00,2022-06-10 18:04:08.584+00,wks-6sdnjgqfpv98,flt-6sdnjgqkpny5,dev-6sdvl6005smx,data,37,"[[0, 4, 0, 0, 0, 0, 1088, ""FF:FF:FF:FF:FF:FF"",...",-2936,1654884230
1,2022-06-10 18:04:45+00,2022-06-10 18:05:04.592+00,wks-6sdnjgqfpv98,flt-6sdnjgqkpny5,dev-6sdvl6005smx,data,35,"[[0, 4, 0, 0, 0, 314, 38896, ""6605471b4464bad4...",-2491,1654884285
2,2022-06-10 18:05:05+00,2022-06-10 18:05:23.549+00,wks-6sdnjgqfpv98,flt-6sdnjgqkpny5,dev-6sdvl6005smx,data,35,"[[0, 4, 0, 0, 0, 0, 10016, ""FF:FF:FF:FF:FF:FF""...",-2133,1654884305
3,2022-06-10 18:05:25+00,2022-06-10 18:05:46.309+00,wks-6sdnjgqfpv98,flt-6sdnjgqkpny5,dev-6sdvl6005smx,data,41,"[[0, 4, 0, 0, 0, 0, 32784, ""FF:FF:FF:FF:FF:FF""...",-2876,1654884325
4,2022-06-10 18:05:45+00,2022-06-10 18:06:05.208+00,wks-6sdnjgqfpv98,flt-6sdnjgqkpny5,dev-6sdvl6005smx,data,42,"[[0, 4, 0, 0, 0, 0, 3888, ""FF:FF:FF:FF:FF:FF"",...",-3130,1654884345


In [5]:
df['scans'][0]

'[[0, 4, 0, 0, 0, 0, 1088, "FF:FF:FF:FF:FF:FF", "35421a8b4c93a349361c390c9e10f2a8faa30641dfa8fdaeaf58cc11ea50f2ad", "FF:FF:FF:FF:FF:FF", "ffe3ac6bffb646885937f662b9abc19a31a38a7501cc27bba3b822fc6b574fc9", -81, 6, 120, null], [0, 5, 0, 0, 2, 48, 12320, "35421a8b4c93a349361c390c9e10f2a8faa30641dfa8fdaeaf58cc11ea50f2ad", "83a0ff7dbfe5db32211647dba4738262e93f751c14332c8f1461142b6e38cbb4", "83a0ff7dbfe5db32211647dba4738262e93f751c14332c8f1461142b6e38cbb4", "9ad78aebdbbff44a53e9c94d2cd5c2782593fd05f3814b65e6367fb5b0ed6520", -85, 6, 204, null], [0, 5, 0, 0, 2, 48, 12320, "35421a8b4c93a349361c390c9e10f2a8faa30641dfa8fdaeaf58cc11ea50f2ad", "83a0ff7dbfe5db32211647dba4738262e93f751c14332c8f1461142b6e38cbb4", "83a0ff7dbfe5db32211647dba4738262e93f751c14332c8f1461142b6e38cbb4", "8034ab62303ccf3ebdc616d5de8d32f76b55b7d3eb26fa4184a0a2320142b950", -85, 6, 204, null], [0, 4, 0, 0, 0, 0, 1104, "FF:FF:FF:FF:FF:FF", "35421a8b4c93a349361c390c9e10f2a8faa30641dfa8fdaeaf58cc11ea50f2ad", "FF:FF:FF:FF:FF:FF", "f

In [6]:
from parse_raw_data import parse_scan_list

test = parse_scan_list(df['scans'][0])
len(test), test[0]

(37,
 [0,
  4,
  0,
  0,
  0,
  0,
  1088,
  '"FF:FF:FF:FF:FF:FF"',
  '"35421a8b4c93a349361c390c9e10f2a8faa30641dfa8fdaeaf58cc11ea50f2ad"',
  '"FF:FF:FF:FF:FF:FF"',
  '"ffe3ac6bffb646885937f662b9abc19a31a38a7501cc27bba3b822fc6b574fc9"',
  -81,
  6,
  120,
  nan])

In [8]:
test[1]

['[0',
 5,
 0,
 0,
 2,
 48,
 12320,
 '"35421a8b4c93a349361c390c9e10f2a8faa30641dfa8fdaeaf58cc11ea50f2ad"',
 '"83a0ff7dbfe5db32211647dba4738262e93f751c14332c8f1461142b6e38cbb4"',
 '"83a0ff7dbfe5db32211647dba4738262e93f751c14332c8f1461142b6e38cbb4"',
 '"9ad78aebdbbff44a53e9c94d2cd5c2782593fd05f3814b65e6367fb5b0ed6520"',
 -85,
 6,
 204,
 nan]

In [7]:
interesting_data = df[['ts', 'device_id', 'scans']]
# we want to change the granularity of the data, i.e., we want a row for each 
# device scanned instead of a row for each scan
new_data = []
for i, row in interesting_data.iterrows():
    dev_id = row.device_id
    ts = row.ts
    scans = parse_scan_list(row.scans)  # this is a list of scans (lists)
    for scan in scans:
        new_data.append([ts, dev_id] + scan)

new_df = pd.DataFrame(new_data)
new_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,1654884230,dev-6sdvl6005smx,0,4,0,0,0,0,1088,"""FF:FF:FF:FF:FF:FF""","""35421a8b4c93a349361c390c9e10f2a8faa30641dfa8f...","""FF:FF:FF:FF:FF:FF""","""ffe3ac6bffb646885937f662b9abc19a31a38a7501cc2...",-81,6,120,NaN
1,1654884230,dev-6sdvl6005smx,[0,5,0,0,2,48,12320,"""35421a8b4c93a349361c390c9e10f2a8faa30641dfa8f...","""83a0ff7dbfe5db32211647dba4738262e93f751c14332...","""83a0ff7dbfe5db32211647dba4738262e93f751c14332...","""9ad78aebdbbff44a53e9c94d2cd5c2782593fd05f3814...",-85,6,204,NaN
2,1654884230,dev-6sdvl6005smx,[0,5,0,0,2,48,12320,"""35421a8b4c93a349361c390c9e10f2a8faa30641dfa8f...","""83a0ff7dbfe5db32211647dba4738262e93f751c14332...","""83a0ff7dbfe5db32211647dba4738262e93f751c14332...","""8034ab62303ccf3ebdc616d5de8d32f76b55b7d3eb26f...",-85,6,204,NaN
3,1654884230,dev-6sdvl6005smx,[0,4,0,0,0,0,1104,"""FF:FF:FF:FF:FF:FF""","""35421a8b4c93a349361c390c9e10f2a8faa30641dfa8f...","""FF:FF:FF:FF:FF:FF""","""ffe3ac6bffb646885937f662b9abc19a31a38a7501cc2...",-80,6,120,NaN
4,1654884230,dev-6sdvl6005smx,[0,5,0,0,2,48,12336,"""35421a8b4c93a349361c390c9e10f2a8faa30641dfa8f...","""83a0ff7dbfe5db32211647dba4738262e93f751c14332...","""83a0ff7dbfe5db32211647dba4738262e93f751c14332...","""99ebc48441ab572335d13ecef9bf272aa752c9b9a90cb...",-85,6,204,NaN
